# Получение ключевых слов для Проектного практикума - Учебная задача (семестр 3)

Черновое решение

## Установка зависимостей (если не установлены ранее)

In [1]:
# Установка PyTorch для ускорителей CUDA
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [2]:
# Установка зависимостей для модели
# %pip install transformers sentencepiece 

In [3]:
# Установка зависимостей для параллельной обработки и блокнота
# %pip install pandas ipywidgets jupyter IProgress protobuf joblib tdqm

## Импорты, константы и настройки

In [4]:
# Импорт библиотек
from os.path import join as path_join
from os.path import exists as path_exists
from os import makedirs

from gc import collect as gc_collect

from torch.cuda import is_available as cuda_is_available
from torch.cuda import device_count as cuda_device_count
from torch.cuda import get_device_name as cuda_get_device_name
from torch.cuda import empty_cache as cuda_empty_cache
from torch import no_grad
from torch import device

import pandas as pd

from joblib import Parallel, delayed
from tqdm import tqdm

from itertools import groupby
from transformers import T5ForConditionalGeneration, T5Tokenizer, BatchEncoding

In [5]:
# Константы 
KW_MAX_LENGTH: int = 64
KW_TOP_P: float = 1.0

PART_FROM: int = 100_000    # Чать от индекса
PART_TO: int = 100_100      # Чать до индекса
PACK_SIZE: int = 500        # Сохранять по

N_JOBS: int = 1        # Тут лучше не переусердствовать, 
BATCH_SIZE: int = 4    # так как VRAM может не хватить

MODEL_NAME: str = '0x7194633/keyt5-large'

PATH_TO_DS: str = '.././data/cleared/'      # '..' из-за того, что блокнот не руте
DS_FILENAME: str = 'cleared_dataset.csv'

PATH_TO_KW_PARTS: str = '.././data/cleared/key_words/'  # '..' из-за того, что блокнот не руте
NAME_OF_KW_PARTS: str = 'key_words'

In [6]:
# Вывод устройств CUDA
for c in range(cuda_device_count()):
    name = cuda_get_device_name(c)
    print(f'cuda:{c} - {name}')

cuda:0 - NVIDIA GeForce RTX 3060 Laptop GPU


In [7]:
# Настройка ускорителя CUDA
CUDA_DEVICE_INDEX: int = 0

In [8]:
# Создание структуры каталогов для данных
def create_paths(fullpath: str) -> bool:
    '''Создание каталогов с подкаталогами'''
    makedirs(fullpath, exist_ok=True)
    return path_exists(fullpath)

print(f'Путь {PATH_TO_KW_PARTS} создан/существует: {create_paths(PATH_TO_KW_PARTS)}')

Путь .././data/cleared/key_words/ создан/существует: True


## Чтение датасета

In [9]:
#Чтение датасета
df_data: pd.DataFrame = pd.read_csv(
    path_join(PATH_TO_DS, DS_FILENAME),
    sep=';',
    index_col='indx')
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499800 entries, 0 to 499999
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   rating_int    499800 non-null  int64 
 1   rubrics_list  499800 non-null  object
 2   cleared_text  499800 non-null  object
dtypes: int64(1), object(2)
memory usage: 15.3+ MB


## Получение ключевых слов

In [10]:
# Функции получения ключевых слов из модели
def get_key_words_batch(
        texts: pd.Series, 
        batch_size: int,
        torch_device: device,
        tokenizer: T5Tokenizer,
        model: T5ForConditionalGeneration,
        **kwargs
        ) -> list[tuple[int, str]]:
    '''Пакетный запрос ключевых слов в модели'''
    # Получаем стартовый индекс (для сохранения индексов)
    indx_list: list[int] = list(texts.index)\
    
    # Деление на пакеты
    result: list[tuple[int, str]] = []
    for batch in range(0, len(texts), batch_size):
        inputs: BatchEncoding = tokenizer(
            texts[batch:batch+batch_size].to_list(), 
            return_tensors='pt',
            padding='longest'
            ).to(torch_device)
        
        with no_grad():
            hypotheses = model.generate(**inputs, num_beams=5, **kwargs).to(torch_device)

        decodeds: list[str] = tokenizer.batch_decode(hypotheses, skip_special_tokens=True)
        
        # Парсинг списка ключевых слов
        for indx, decoded in zip(indx_list[batch:batch+batch_size], decodeds):
            decoded_list: list[str] = decoded.replace('; ', ';') \
                .replace(' ;', ';').lower().split(';')[:-1]
            decoded_list: list[str] = [el for el, _ in groupby(decoded_list)]
            row: tuple[int, str] = (indx, decoded_list)
            result.append(row)
    return result

def get_key_words(texts: pd.Series) -> pd.Series:
    '''Получение ключевых слов для массива текстов'''
    if not cuda_is_available():
        raise RuntimeError('Ускоритель CUDA не доступен! Вычисление на CPU может занять годы')
    torch_device: device = device(f'cuda:{CUDA_DEVICE_INDEX}')
    print(f'Расчеты будут вестись на: {torch_device} ({cuda_get_device_name(torch_device)})')

    # Создание инстанции модели
    tokenizer: T5Tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, legacy=False)
    model: T5ForConditionalGeneration = T5ForConditionalGeneration \
        .from_pretrained(MODEL_NAME) \
        .to(torch_device)
    
    try:
        result: list[tuple[int, str]] = get_key_words_batch(
            texts,
            BATCH_SIZE,
            torch_device,
            tokenizer, 
            model, 
            top_p=KW_TOP_P, 
            max_length=KW_MAX_LENGTH)
    except Exception as ex:
        raise Exception(ex)
    finally:
        # Чистка памяти (иначе видеопамять кончится)
        cuda_empty_cache()
        del model, tokenizer, torch_device
        gc_collect()
    
    # Сборка серии
    idx = [x[0] for x in result]
    vals = [x[1] for x in result]
    return pd.Series(vals, index=idx)

In [11]:
# Датасет оптимально делить на части и вычислять на разных PC
df_part = df_data[PART_FROM:PART_TO]
parts: list[(int, int, pd.DataFrame)] = []

for pack in range(0, len(df_part), PACK_SIZE):
    df_pack = df_part[pack:pack+PACK_SIZE]
    parts.append((PART_FROM+pack, PART_FROM+pack+len(df_pack), df_pack))

def part_calc(f: int, t: int, df_data: pd.DataFrame) -> pd.Series:
        '''Получение и сохранение ключевых слов'''
        try:
            print(f'-----> From {f} To {t}')
            result_ser: pd.Series = get_key_words(df_data['cleared_text'])
            result_ser.to_csv(
                path_join(PATH_TO_KW_PARTS, f'{NAME_OF_KW_PARTS}_{f}_{t}.csv'),
                encoding='utf-8-sig',
                sep=';',
                index_label='indx')
            return result_ser
        except Exception as ex:
            print(ex)

In [12]:
# Минимизация памяти для ускорения передачи
# при параллельном вычислении
del df_data, df_part, df_pack
gc_collect()

0

In [13]:
# Запуск получения ключевых слов в потоках или в одном
if N_JOBS < 2:
    for f, t, df in tqdm(parts):
        part_calc(f, t, df)
else:
    Parallel(
        n_jobs=N_JOBS,
        # backend="threading",
        # backend="multiprocessing",
        verbose=1)(delayed(part_calc) \
                (f, t, df) for f, t, df in tqdm(parts))

  0%|          | 0/1 [00:00<?, ?it/s]

-----> From 100000 To 100100
Расчеты будут вестись на: cuda:0 (NVIDIA GeForce RTX 3060 Laptop GPU)


100%|██████████| 1/1 [00:54<00:00, 54.58s/it]
